In [2]:
import sys
import os

ROOT_DIR = os.path.normpath(os.getcwd() + os.sep + os.pardir + os.sep + os.pardir) #change it to dir where Data folder is 
path=os.path.join(ROOT_DIR,'Data')

datafile='eda_final_df.csv'# use the filename that your data file is named.
PATH_TO_DATA=os.path.join(path,datafile)

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

from recomm_sys_v1_helper import rows_to_del_with_index
from recomm_sys_v1_helper import top_merch
from recomm_sys_v1_helper import merchCleanup
from recomm_sys_v1_helper import createCatDF
from recomm_sys_v1_helper import takeInputMerch
from recomm_sys_v1_helper import filters
from recomm_sys_v1_helper import recommendationSystem_A
from recomm_sys_v1_helper import recommendationSystem_B
from recomm_sys_v1_helper import recommendationSystem_C


In [4]:
#Thresholds for filtering results from the recommendation engine

Threshold_Sim_Score = 0.3 #tolerance fraction of accepting similarities among merchants. (0=too tolerant (accept all), 1: no tolerance(accept none))
Threshold_Num_Visit = 6 #minimum of 6 visits must be counted by combined users before recommending merchant
Threshold_Num_MeanAmt = 10 #minimum average of $10 must be spent by combined users before recommending merchant

#### Reading CSV file of the dataframe created at the end of EDA (eda_df.csv)

In [5]:
df1_rec=pd.read_csv(PATH_TO_DATA)
# Reseting index to integer from datatime
df1_rec=df1_rec.reset_index(drop=True)
#Changing name of col 'simple_description' to 'merchant'
df1_rec.rename(columns={"simple_description": "merchant"}, inplace=True) 

In [6]:
df1_rec

,uid,category,categoryid,merchant,amountnum,city,state,zip_code,address,latitude,longitude,accountid
0,WdtUjyA3mghZ6TrphronQpCVrI23,Entertainment,7.00,sirius xm,20.63,NaN,NaN,NaN,NaN,nan,nan,"10,038,937.00"
1,WdtUjyA3mghZ6TrphronQpCVrI23,Restaurants/Dining,22.00,*pending debit aug20 p 2818at buona glen ellyn...,20.69,Glen Ellyn,NaN,NaN,NaN,nan,nan,"10,038,937.00"
2,WdtUjyA3mghZ6TrphronQpCVrI23,Groceries,10.00,the beer cellar,28.51,Glen Ellyn,NaN,NaN,NaN,nan,nan,"10,038,937.00"
3,WdtUjyA3mghZ6TrphronQpCVrI23,Entertainment,7.00,disney plus,6.99,NaN,CA,NaN,NaN,nan,nan,"10,038,937.00"
4,WdtUjyA3mghZ6TrphronQpCVrI23,Restaurants/Dining,22.00,p #2818christ cafe 95 oak lawn il x0232,1.09,Oak Lawn,IL,NaN,NaN,nan,nan,"10,038,937.00"
...,...,...,...,...,...,...,...,...,...,...,...,...
52257,gg95CpaLypcaAzGvF9fGqmF6W2w1,Clothing/Shoes,5.00,nordstrom rack,90.04,Westminster,NaN,NaN,NaN,nan,nan,"10,009,118.00"
52258,gg95CpaLypcaAzGvF9fGqmF6W2w1,General Merchandise,44.00,costco,73.42,Thornton,CO,80233,NaN,nan,nan,"10,009,118.00"
52259,qy4wWJHVBMYbF3GnaQKCAsMej6k2,Entertainment,7.00,itunes store,5.36,NaN,NaN,NaN,NaN,nan,nan,"10,019,665.00"
52260,qy4wWJHVBMYbF3GnaQKCAsMej6k2,Clothing/Shoes,5.00,2ohio state u 9x8240columbus oh xx4798,16.00,Columbus,OH,NaN,NaN,nan,nan,"10,019,665.00"


### Cleaning some obvious "oops" in merchant (aka simple_description) field like the word pending or visa infront of merchant names etc.

In [7]:
rest_list=[]
for merch in df1_rec.merchant:
    if ((merch[0:8]=='*pending')):
        rest_list.append(merch[30:])
    elif (merch[0:7]=='pending'):
        rest_list.append(merch[23:])
    elif (merch[0:2]=='p '):
        rest_list.append(merch[11:25])
    elif (merch[0:2]=='pp'):
        rest_list.append(merch[3:])
    elif (merch[0:2]=='a '):
        rest_list.append(merch[7:])
    elif (merch[0:8]=='visa pp*'):
        rest_list.append(merch[8:])
    elif (merch[0:7]=='visa sq'):
        rest_list.append(merch[9:])
    elif (merch[0:4]=='visa'):
        rest_list.append(merch[5:26])
    elif (merch[0:2]=='sq'):
        rest_list.append(merch[4:])
    elif (merch[0:2]=='sp'):
        rest_list.append(merch[5:])
    else:
        rest_list.append(merch)
        
#rest_list #visa visa pp* 'sq, visa jefes tacos & tequila 'yonah mountain fridas mexican zucca bar ziggy tandur indi, not just coffe, winot coffee, christ cafe
sorted(rest_list)
df1_rec.reset_index(inplace=True, drop=True)
df1_rec['merchant']=rest_list

#### Dropping some more 'merchants' as those look like mistakes or not real merchants.

In [8]:
#Deleting all the merchant records from all of the categories that are not needed

deleteList=['ssp*rockymoutnain kung fuxx2839 co', 'brewing *******et', 'bethel university','apple.com/bill',\
            'amazon *******etplace','amazon.com','amazon prime video',\
            'ach transaction - qapital transfer 0007392303', 'ach transaction - qapital invest l ach 0007100028',\
            'direct withdrawal','travel','qoins','venmo','porkbun com portland us', 'otis craft collect lafayette us',\
            'live your life integratedxx5229 us', 'paypal','webflow com', 'visible service llc','nzb.su httpsnzb.su fl',\
            'premier1 supplies xx7622 ia','premier1 supplies xx7622 us','home depot',"fisk's 'farm & home suppl decorah ia",\
            "dbt mcguckin hardwar |mcguckin hardware boulder co us|card nbr: 0853",'flying b bar ranch llc xx06 co',\
            'front range eye health louisville co', 'mini miners pediatric erie us','chpg primary care erie erie co',\
            'mini miners pediatric denerie co','lombard direct']

df1_rec=merchCleanup(df1_rec,deleteList)

In [9]:
dict_startswith={'costco gas stations':'costco gas station',"\'circle k']":'circle k'}
for k,v in dict_startswith.items():
    df1copy=df1_rec.copy()
    mask=(df1copy.merchant.str.startswith(k))
    df1copy['merchant']=np.where(mask,v,df1copy.merchant)
    df1_rec=df1copy

In [10]:
df1_rec.info() #48811 non-null entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48811 entries, 0 to 48810
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   uid         48811 non-null  object 
 1   category    48811 non-null  object 
 2   categoryid  48811 non-null  float64
 3   merchant    48811 non-null  object 
 4   amountnum   48811 non-null  float64
 5   city        19808 non-null  object 
 6   state       24375 non-null  object 
 7   zip_code    12974 non-null  object 
 8   address     8959 non-null   object 
 9   latitude    9061 non-null   float64
 10  longitude   9061 non-null   float64
 11  accountid   48811 non-null  float64
dtypes: float64(5), object(7)
memory usage: 4.5+ MB


#########################################################################################################################

## Goal # 1: Collaborative Filtering. 


## Recommending merchants based on other similar merchants. 

I will focus on extracting merchants from one category at a time and focusing only on top 10-15 (per Clair's advise) merchants/category

In [11]:
#Finding what are the top categories (with >900 rows)
df1_rec.groupby(['category'])['category'].count().sort_values(ascending=False)

category
Restaurants/Dining          8261
Shops                       6021
General Merchandise         5498
Groceries                   4707
Service                     4064
Travel                      3662
Food and Drink              2836
Clothing/Shoes              1749
Entertainment               1576
Online Services             1522
Gasoline/Fuel               1347
Home Improvement            1091
Healthcare/Medical           970
Personal Care                656
Electronics                  583
Utilities                    566
Hobbies                      556
Service Charges/Fees         352
Automotive Expenses          341
Telephone Services           317
Cable/Satellite Services     315
Pets/Pet Care                302
Community                    255
Education                    223
Recreation                   220
Postage and Shipping         199
Dues and Subscriptions       176
Home Maintenance             111
Gifts                        105
Healthcare                    98
O

#### The recommendation system code here is only coded for and tested for categories with > 900 records except for Online Services:

#### Restaurants/Dining, Shops, General Merchandise, Service, Groceries, Travel, Food and Drink, Clothing/Shoes, Entertainment,      Gasoline/Fuel, Home Improvement, Healthcare/Medical

In [12]:
#creating dictionary with category as Key and categoryid as Value.
cat_catid_dict=df1_rec[['categoryid','category']].groupby('category')['categoryid'].first().to_dict()
{k: v for k, v in sorted(cat_catid_dict.items(), key=lambda item: item[1])}

{'Automotive Expenses': 2.0,
 'Child/Dependent Expenses': 4.0,
 'Clothing/Shoes': 5.0,
 'Education': 6.0,
 'Entertainment': 7.0,
 'Gasoline/Fuel': 8.0,
 'Gifts': 9.0,
 'Groceries': 10.0,
 'Healthcare/Medical': 11.0,
 'Home Maintenance': 12.0,
 'Home Improvement': 13.0,
 'Cable/Satellite Services': 15.0,
 'Online Services': 16.0,
 'Personal Care': 20.0,
 'Restaurants/Dining': 22.0,
 'Travel': 23.0,
 'Service Charges/Fees': 24.0,
 'Hobbies': 34.0,
 'Telephone Services': 38.0,
 'Utilities': 39.0,
 'Pets/Pet Care': 42.0,
 'Electronics': 43.0,
 'General Merchandise': 44.0,
 'Office Supplies': 45.0,
 'Advertising': 100.0,
 'Postage and Shipping': 104.0,
 'Dues and Subscriptions': 108.0,
 'Shops': 201.0,
 'Service': 202.0,
 'Food and Drink': 203.0,
 'Healthcare': 204.0,
 'Community': 205.0,
 'Recreation': 206.0}

### 1. Selecting  a category to evaluate the recommendation engine code
For now I am taking input to a "currentCat" variable to determine which category code needs to be executed.
Default category is "Resturants/Dining"

In [13]:
currentCat=input('Category?') #This will come from outside of this code!

if currentCat=='': #if left empty code will run for resturant/dining by default
    currentCatid=22.0
    currentCat='Restaurants/Dining'
else:
    currentCatid=cat_catid_dict[currentCat] #getting value with the key assigned to variable currentCat from cat_catid_dict created above
currentCatid

Category?


22.0

#### Display top merchant list of the current category id

In [14]:
#Current top 15 merchants in the selected category (using currentCatid)
#This cell shows what top15 looks like before deleting some that don't make sense in the next step
print("______________________________________________________________")
print("Showing top merchants from category ---  {}".format(currentCat))
print("______________________________________________________________")
top_merch_list=top_merch(df1_rec,currentCatid,15)
top_merch_list

______________________________________________________________
Showing top merchants from category ---  Restaurants/Dining
______________________________________________________________


['starbucks',
 "dunkin' donuts",
 'chipotle mexican grill',
 'chick-fil-a',
 'firehouse subs',
 "mcdonald's",
 'sushi aji',
 "wendy's",
 'morning glory cafe',
 'taco bell',
 'christi cafe',
 'mici handcrafted italian',
 'subway',
 'ziggis coffee',
 'pret a manger']

#### 2.Creating DF of all the <u>remaining</u> records of the current chosen cateogry
##### (df1_rec_category)

In [15]:
df1_rec_category=createCatDF(df1_rec, currentCatid, top_merch_list)

#### 3.Collecting(input) a merchant from selected category for checking model accuracy 
##### (merchant_user_visited)

In [16]:
 merchant_user_visited=takeInputMerch(currentCatid,df1_rec_category,top_merch_list)

Please select one of the merchants from this list. 
['starbucks', "dunkin' donuts", 'chipotle mexican grill', 'chick-fil-a', 'firehouse subs', "mcdonald's", 'sushi aji', "wendy's", 'morning glory cafe', 'taco bell', 'christi cafe', 'mici handcrafted italian', 'subway', 'ziggis coffee', 'pret a manger']


#### 4. Creating filters 
##### (count_per_merch) 
(1) popularity (number of visits by users) of other similar merchants

##### (meanamt_per_merch)
(2) mean amount (dollars) spent by other users at the similar merchants 

In [17]:
count_per_merch, meanamt_per_merch = filters(currentCatid,df1_rec_category, top_merch_list)

#### 5. Create sparse matrix of all user ids (uid) and merchants (merchants) of the specific category 
##### (df1_rec_merch_count_mtx)

In [18]:
#SparceMatrix
df1_rec_merch_count_mtx=df1_rec_category.pivot_table(index='uid',columns='merchant', values='amountnum',\
                                                         aggfunc='count', fill_value=0)

#### 6. Create a series showing number of visits users made to the current merchant(merchant_popularity_count)

In [19]:
#Merchant populaity matrix
merchant_popularity_count=df1_rec_merch_count_mtx[merchant_user_visited] #How many times each user visited the "merchant_user_visited"

### 8. Building Model

### Evaluating Similarity Based on Pearson Correlation OR Cosine Similarity Score

#### 1 (a) . Compute Pearson Correlation
Using Pandas 'dataframe.corrwith()' method compute pairwise correlation between rows of two dataframes, df1_rec_merch_count_mtx and 'merchant_popularity_count'. This will produce a series with numbers between [-1,1], showing correlation between all the merchants and the merchant_user_visited. Closer the score is to 1 or (-1), greater is the correlation between the two entities. 

#### 1 (b) . Compute Cosine Similarity 
Using sklearn.metrics.pairwise submodule's function, cosine_similarity, compute similarities between all the merchants. The result is a nxn sparse matrix, where n is total number of merchants in a refined category data. A number closer to 1 shows high similarity and a number close to 0 shows less similarity between the merchants.

#### 2. Filering
Using Thresholds for Pearson Score, along with Thresholds for the filters created before (count_per_merch and meanamt_per_merch), filtter out the rows (merchants) that fall within those boundaries. 

#### 3. Recommendations:
        (1) Recommend up to top 3 merchants that are most similar to the current merchant.

### 1. Pearson Similarity Results Only

In [20]:
recommendationSystem_A(merchant_user_visited,'Pearson',df1_rec_merch_count_mtx,merchant_popularity_count,
                     count_per_merch, meanamt_per_merch,Threshold_Sim_Score, Threshold_Num_MeanAmt, Threshold_Num_Visit,top_merch_list)

Other merchants you may like ["wendy's", 'chick-fil-a', 'taco bell']


### 2. Cosine Similarity Results Only

In [21]:
recommendationSystem_A(merchant_user_visited,'Cosine',df1_rec_merch_count_mtx,merchant_popularity_count, \
                     count_per_merch, meanamt_per_merch, Threshold_Sim_Score, Threshold_Num_MeanAmt, \
                     Threshold_Num_Visit,top_merch_list)

Other merchants you may like ['starbucks', 'subway', 'taco bell']


### 3. Combined logic Using Cosine And Pearson Similarity

#### 1 . Compute Cosine and Pearsom Similarity 
Both Cosine and Pearson Similarities are calculated. The final recommendation are devised based on:

.If cosine similarity resulted in 3 or more merchants --> Show recommendations from cosine similarity results

.If cosine similarity resuled in < 3 and pearson similarity resulted in < 3 merchants --> Show recommendations by join both results 

.If cosine similarity resulted in 0 merchants --> Show recommendations from pearson similarity results

#### Note: filters count_per_merch and meanamt_per_merch will remain the same and will used in same way as explained before
 

In [22]:
recommendationSystem_B(merchant_user_visited,df1_rec_merch_count_mtx,merchant_popularity_count,count_per_merch,meanamt_per_merch,\
                                          Threshold_Sim_Score,Threshold_Num_MeanAmt,Threshold_Num_Visit,top_merch_list)

Other merchants you may like ['starbucks', 'subway', 'taco bell']


### 4.  Selecting Similarity Method: Cosine or Pearson. 
### Output is based on similarity score only and is in the form of dictionary

##### Per Claire's email 9/29: Adding additional code so that FinGoal have a choise to (1) select Cosine or Pearson (2) In the output only see top 5 merchants based on top 5 similarity scores

In [23]:
#Choose a similarity method

SimMethod=input('Similarity Method? Cosine or Pearson'+ '\n').lower() # if left empty, default selection is Pearson

recommendationSystem_C(merchant_user_visited,SimMethod,df1_rec_merch_count_mtx,merchant_popularity_count,count_per_merch, meanamt_per_merch,top_merch_list)

Similarity Method? Cosine or Pearson

Merchant User Visited: mcdonald's


For the Similarity Method of: pearson

Top merchants are: 


{"wendy's": 0.7668126770783177,
 'ziggis coffee': 0.6867628675074063,
 'chick-fil-a': 0.6636641147708083,
 'taco bell': 0.5551009820662435,
 'chipotle mexican grill': 0.2789634867651392}

### The End!